**TODO:**
- Look into further ways we can play with metadata
- Add structured data portion

# Retrieval Augmented Generation with Unstructured Data using GPUs

## Setup

Using the command line, create a new Conda environment using the `environment.yml` file:
```bash
module load miniforge
conda env create -f environment.yml
conda activate rag_ollama
```

Alternatively, install the necessary packages manually:

```bash
module load miniforge
conda create -n rag_ollama jupyterlab langchain-ollama langchain-chroma langchain-community
conda activate rag_ollama
pip install "unstructured[pdf]"
```

Create a Jupyter kernel for your environment:
```bash
python -m ipykernel install --user --name rag_ollama
```

Connect this notebook to the Jupyter kernel you just created. You may need to disconnect from and reconnect to your Jupyter session.

Run the setup script to start Ollama and download the embedding and language models:
```bash
sh start_ollama.sh
```

Import packages:

In [1]:
import os
import json
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings, OllamaLLM
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.agents.middleware import dynamic_prompt, ModelRequest
from langchain.agents import create_agent
import logging
from IPython.display import Markdown, display

/home/secorey/orcd/scratch/.conda/envs/rag_ollama/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Set environment variables:

In [2]:
with open("config.json", "r") as f:
    config = json.load(f)

EMBEDDING_MODEL_NAME = config["embedding_model"]
LLM_NAME = config["llm"]

DOCS_PATH = os.path.join(os.getcwd(), "docs")
VECTOR_STORE_PATH = os.path.join(os.getcwd(), "vector_store")

Initialize components:

In [3]:
embeddings = OllamaEmbeddings(
    model=EMBEDDING_MODEL_NAME
)

## Processing PDFs

We have a set of PDFs that we would like to input into our RAG pipeline. We cannot do this directly, however. While PDFs are optimized for humans to read and comprehend, machines have a harder time. So, we must first process our documents so that they can be efficiently searched by a computer. We will do this in two steps:
1. Extract the raw text from the PDFs
2. Convert the text into vectors using an embedding model

### Extracting Text from PDFs

The `unstructured` software has a PDF loading tool that extracts text from PDFs and ignores images. This software uses the `pdfminer.six` Python package under the hood, which is very popular for reading PDFs using Python.

*Note: `unstructured` has loaders for other file formats as well, such as Markdown or Word documents.*

In [4]:
# Mute pdfminer warnings globally
logging.getLogger("pdfminer").setLevel(logging.ERROR)
logging.getLogger("pdfminer.pdffont").setLevel(logging.ERROR)

def load_documents(docs_path):
    """
    Load documents from the specified directory recursively. Documents must be
    in .pdf format.
    """

    # Load the documents recursively:
    documents = []
    for file_name in os.listdir(docs_path):
        file_path = os.path.join(docs_path, file_name)
        if file_name.endswith('.pdf'):
            loader = UnstructuredPDFLoader(file_path, languages=["eng"])
            doc = loader.load()
            doc[0].metadata["source"] = file_name
            documents.extend(doc)
        elif os.path.isdir(file_path):
            documents.extend(load_documents(file_path))
    return documents

documents = load_documents(DOCS_PATH)

This creates a list of documents:

In [5]:
documents[0]

Document(metadata={'source': 'CUBIC-checkpoint.pdf'}, page_content='CUBIC: A New TCP-Friendly High-Speed TCP Variant ∗\n\nSangtae Ha, Injong Rhee Dept of Computer Science North Carolina State University Raleigh, NC 27695 {sha2,rhee}@ncsu.edu\n\nLisong Xu Dept of Comp. Sci. and Eng. University of Nebraska Lincoln, Nebraska 68588 xu@cse.unl.edu\n\nABSTRACT CUBIC is a congestion control protocol for TCP (transmis- sion control protocol) and the current default TCP algo- rithm in Linux. The protocol modiﬁes the linear window growth function of existing TCP standards to be a cubic function in order to improve the scalability of TCP over fast and long distance networks. It also achieves more eq- uitable bandwidth allocations among ﬂows with diﬀerent RTTs (round trip times) by making the window growth to be independent of RTT – thus those ﬂows grow their conges- tion window at the same rate. During steady state, CUBIC increases the window size aggressively when the window is far from the satu

Each document object contains metadata, such as the document title, as well as the raw text.

### Creating a Vector Store

Now that we have extracted the text from the PDFs, we must further process our data so that it can be efficiently searched by our pipeline. We will do this by saving our documents into a **vector store**.

#### Chunking

The vectors will be created using an embedding model, but before we do this, we must **chunk** our documents. We have to do this because embedding models have a context limit, and some of our documents are too large to fit into a single vector. For example, the embedding model that we're using, `bge-m3`, has a context limit of 8,000 tokens (per its [datasheet](https://ollama.com/library/bge-m3)).

How you chunk your documents is important because each chunk should represent a coherent idea that reflects the intended meaning from the original document. If your chunks are too large, you risk feeding your pipeline unnecessary or only tangentially relevant information. If your chunks are too small, then you may lose essential context that helps the retriever and model understand what a chunk is actually about. Consider this example:

> Red squirrels have a varied and adaptable diet that changes with the seasons. They primarily eat seeds from conifer cones, such as pine, spruce, and fir, carefully stripping the cones to reach the nutritious seeds inside. In addition to seeds, they consume nuts, berries, fruits, buds, and fungi, especially mushrooms. Red squirrels are also known to occasionally eat insects, bird eggs, or nestlings when plant food is scarce.

> Red squirrels typically live in forests dominated by coniferous or mixed trees, which provide both food and shelter. They build nests, called dreys, high in the trees using twigs, leaves, moss, and bark for insulation. Some individuals also use hollow trees or abandoned woodpecker holes for nesting. Their habitat usually includes well-defined territories that they actively defend from other squirrels.

If we combine both paragraphs into a single chunk, then a query about the diet of red squirrels will retrieve information about their habitat and nesting behavior as well. While this information is related, it is not directly relevant to the question being asked. As a result, the retrieved context may fill up the model’s context window more quickly and crowd out other, more relevant chunks from different documents.

On the other hand, if we split the document too aggressively (e.g., by making each sentence into its own chunk), then the sentences' original context is lost. Important information that is implicit in the surrounding sentences may no longer be available to the retriever. For example, if a user asks, "What do red squirrels eat?", the retriever may fail to identify the following sentence as relevant:

> They primarily eat seeds from conifer cones, such as pine, spruce, and fir, carefully stripping the cones to reach the nutritious seeds inside.

On its own, this sentence does not explicitly mention red squirrels. Without the surrounding context, the retriever (and the model) has no clear signal that the sentence is describing the diet of red squirrels rather than some other animal.

In this example, the best method would be to treat each paragraph as its own chunk, as each paragraph has a distinct topic.

Of course, we cannot manually chunk every document. Instead, chunking tools allow us to specify chunk sizes, and also include chunk overlaps, which help avoid context loss. Here, I've specified 1200 characters, which is about the length of a short paragraph.

In [7]:
chunk_size = 10000
chunk_overlap = int(.2 * chunk_size)
# separators=["\n\n", "\n", ". ", " ", ""]
separators=["\n\n", "\n", ". "]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, # Given in characters, not tokens (1 token = 3-4 characters)
    chunk_overlap=chunk_overlap,
    length_function=len,
    separators=separators,
)
split_docs = text_splitter.split_documents(documents)
print(f"Loaded {len(documents)} docs -> split into {len(split_docs)}")

Loaded 6 docs -> split into 39


After splitting, we have multiple documents, each representing a different chunk of each source:

In [8]:
for doc in split_docs:
    print(doc.metadata["source"])

CUBIC-checkpoint.pdf
CUBIC-checkpoint.pdf
CUBIC-checkpoint.pdf
CUBIC-checkpoint.pdf
CUBIC-checkpoint.pdf
CUBIC-checkpoint.pdf
CUBIC-checkpoint.pdf
BBR.pdf
BBR.pdf
BBR.pdf
BBR.pdf
BBR.pdf
BBR.pdf
CUBIC.pdf
CUBIC.pdf
CUBIC.pdf
CUBIC.pdf
CUBIC.pdf
CUBIC.pdf
CUBIC.pdf
Hybla.pdf
Hybla.pdf
Hybla.pdf
Hybla.pdf
Hybla.pdf
Hybla.pdf
Hybla.pdf
Hybla.pdf
NewReno.pdf
NewReno.pdf
NewReno.pdf
NewReno.pdf
NewReno.pdf
NewReno.pdf
comparative_study.pdf
comparative_study.pdf
comparative_study.pdf
comparative_study.pdf
comparative_study.pdf


#### Converting Chunks into Vectors

The final step of data preparation is to convert our documents into "vectors," which are numerical representations that capture the meaning of words, sentences, and passages. This allows the pipeline to efficiently run a similarity search with queries to extract contextually relevant documents.

*How do we choose an embedding model?*

The embedded documents get saved to a Chroma database ("vector store"):

In [10]:
Chroma.from_documents(documents=split_docs,
                      embedding=embeddings,
                      persist_directory=VECTOR_STORE_PATH) # Specifying persist_directory saves the vector store as a file so we don't have to recreate it

Once the vector store has been saved to a file, you can read it using the following command:

In [11]:
vector_store = Chroma(persist_directory=VECTOR_STORE_PATH,
                      embedding_function=embeddings)

Now that we have a vector store, we can evaluate its ability to retrieve relevant information using similarity searches. For example:

In [17]:
def get_relevant_docs(query, vector_store):
    results = vector_store.similarity_search_with_score(
        query, k=5
    )
    for res, score in results:
        print(res.metadata["source"], f"({round(score, 2)})")

query = "How does BBR work?"
get_relevant_docs(query, vector_store)

BBR.pdf (0.76)
BBR.pdf (0.78)
BBR.pdf (0.81)
BBR.pdf (0.84)
BBR.pdf (0.89)


Using a basic query, our retriever seems to work well. Let's try something more complex:

In [23]:
query = "Contrast the strategies used by CUBIC, Hybla, and BBR to handle connections with long Round Trip Times (RTT)."
get_relevant_docs(query, vector_store)

BBR.pdf (0.77)
Hybla.pdf (0.8)
BBR.pdf (0.8)
CUBIC-checkpoint.pdf (0.81)
CUBIC.pdf (0.81)


In [27]:
query = "How do 'loss-based' protocols and 'delay-based' protocols struggle specifically in the context of Low-Earth-Orbit (LEO) satellite networks?"
get_relevant_docs(query, vector_store)

comparative_study.pdf (0.74)
comparative_study.pdf (0.78)
comparative_study.pdf (0.8)
comparative_study.pdf (0.84)
comparative_study.pdf (0.87)


## Running RAG

### Run an LLM locally

In [29]:
llm = OllamaLLM(model=LLM_NAME, temperature=0.5)
display(Markdown(llm.invoke("Where is MIT located?")))

MIT, or the Massachusetts Institute of Technology, is located in Cambridge, Massachusetts, United States. Specifically, its main campus is situated on a 168-acre site along the Charles River, adjacent to Boston and Harvard University.

Here's the exact address:

Massachusetts Institute of Technology
77 Massachusetts Avenue
Cambridge, MA 02139

MIT also has other campuses and facilities located in nearby cities, including Cambridge, Boston, and Lexington, but its main campus is in Cambridge.

In [30]:
display(Markdown(llm.invoke("How do I use Pandas in Python?")))

**Getting Started with Pandas**
=====================================

Pandas is a powerful library for data manipulation and analysis in Python. Here's a step-by-step guide to get you started:

### Installing Pandas

First, make sure you have the latest version of Python installed on your system. Then, open a terminal or command prompt and run:
```bash
pip install pandas
```
This will install the `pandas` library.

### Basic Usage

Here's an example of creating a simple DataFrame (a 2-dimensional labeled data structure with columns of potentially different types):
```python
import pandas as pd

# Create a dictionary with some sample data
data = {'Name': ['John', 'Anna', 'Peter'],
        'Age': [28, 24, 35],
        'Country': ['USA', 'UK', 'Australia']}

# Create a DataFrame from the dictionary
df = pd.DataFrame(data)

print(df)
```
Output:
```
     Name  Age    Country
0    John   28         USA
1    Anna   24          UK
2   Peter   35  Australia
```
### Basic Operations

Here are some basic operations you can perform on a DataFrame:

* **Selecting columns**: `df['Name']` or `df[['Name', 'Age']]`
* **Filtering rows**: `df[df['Age'] > 30]`
* **Sorting data**: `df.sort_values(by='Age')`

### Data Manipulation

Pandas provides various functions for data manipulation, including:

* **GroupBy**: group data by one or more columns and perform aggregation operations
* **Merging**: combine two or more DataFrames based on a common column
* **Reshaping**: change the shape of a DataFrame (e.g., from long to wide)

Here's an example of using GroupBy:
```python
# Create another DataFrame with some sample data
data2 = {'Name': ['John', 'Anna', 'Peter'],
         'Age': [28, 24, 35],
         'Country': ['USA', 'UK', 'Australia'],
         'Score': [90, 80, 95]}

df2 = pd.DataFrame(data2)

# Group by Country and calculate the mean Score
grouped_df = df2.groupby('Country')['Score'].mean()

print(grouped_df)
```
Output:
```
Country
Australia    95.0
UK           80.0
USA          90.0
Name: Score, dtype: float64
```
### Data Analysis

Pandas provides various functions for data analysis, including:

* **Descriptive statistics**: `df.describe()`
* **Correlation matrix**: `df.corr()`

Here's an example of using descriptive statistics:
```python
# Calculate the mean, median, and standard deviation of Age
print(df['Age'].describe())
```
Output:
```
count    3.000000
mean     29.000000
std      5.477225
min      24.000000
25%      27.500000
50%      29.000000
75%      30.500000
max      35.000000
Name: Age, dtype: float64
```
This is just a brief introduction to using Pandas in Python. I hope this helps you get started with data manipulation and analysis!

___

### Set up the RAG pipeline

In [43]:
@dynamic_prompt
def prompt_with_context(request: ModelRequest) -> str:
    """Inject context into state messages."""
    last_query = request.state["messages"][-1].text
    retrieved_docs = vector_store.similarity_search(last_query, k=3)

    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

    # Print documents used:
    print("\nRetrieved documents:") ##
    for doc in retrieved_docs: ##
        print(doc.metadata["source"]) ##
    print()

    system_message = (
        "You are a helpful assistant. Answer only using the information from the following documents."
        f"\n\n{docs_content}"
    )

    return system_message


agent = create_agent(llm, tools=[], middleware=[prompt_with_context])

def pose(query):
    for step in agent.stream(
        {"messages": [{"role": "user", "content": query}]},
        stream_mode="values",
    ):
        display(Markdown(step["messages"][-1].text))
        

Testing questions:

In [39]:
query = "How do the design philosophies of BBR, CUBIC, and NewReno differ in their interpretation of network 'signals'?"
pose(query)

How do the design philosophies of BBR, CUBIC, and NewReno differ in their interpretation of network 'signals'?


Retrieved documents:
BBR.pdf
BBR.pdf
comparative_study.pdf



The design philosophies of BBR (Bottleneck Bandwidth and Round-trip time), CUBIC, and TCP NewReno differ in their interpretation of network "signals" or indicators that a congestion control algorithm uses to adjust its sending rate.

**TCP NewReno:**

* Interprets packet loss as the primary signal for congestion.
* Assumes that packet loss is due to buffer overflow at routers, which indicates that the link bandwidth is fully utilized and the sender should reduce its rate.
* Uses packet loss to trigger a reduction in the congestion window (CWND) and slow-start to recover from congestion.

**CUBIC:**

* Interprets packet loss as a signal for congestion, but also considers other factors such as packet delay and jitter.
* Aims to fill up the link bandwidth without building up a queue, similar to TCP Vegas.
* Uses a more aggressive approach than NewReno, increasing the CWND faster in response to available bandwidth.

**BBR (Bottleneck Bandwidth and Round-trip time):**

* Interprets network "signals" as the bottleneck bandwidth and round-trip time (RTT) of the network path.
* Aims to fill up the link bandwidth without building up a queue, but does so by frequently probing the network for its minimal RTT and bottleneck bandwidth.
* Adjusts the TCP sending rate to match the bandwidth-delay product, which is calculated based on the observed bottleneck bandwidth and RTT.

In summary:

* NewReno focuses on packet loss as the primary signal for congestion.
* CUBIC considers packet loss, delay, and jitter as signals for congestion and aims to fill up link bandwidth without queuing.
* BBR interprets network "signals" as the bottleneck bandwidth and RTT, aiming to match the sending rate with the bandwidth-delay product.

These differences in design philosophies lead to distinct behaviors and performance characteristics of each algorithm in different network conditions.

In [34]:
query = "Why does TCP Vegas perform poorly in LEO satellite networks compared to BBR?"
pose(query)

Why does TCP Vegas perform poorly in LEO satellite networks compared to BBR?


Retrieved documents:
comparative_study.pdf
comparative_study.pdf
comparative_study.pdf



According to the text, TCP Vegas detects congestion by delay increase, which can lead to low throughput because it is sensitive to network delays. In LEO satellite networks, the frequent handovers and varying link delays due to satellite movement can cause TCP Vegas to incorrectly identify these changes as congestion, leading to poor performance.

In contrast, BBR (Bottleneck Bandwidth and Round-trip time) shows more resilient behavior in LEO satellite networks because it frequently probes the network bandwidth and RTT, allowing it to adapt to changing conditions. This enables BBR to achieve a good balance between throughput and latency, even in the face of frequent path changes and delay variations caused by satellite movement.

It's worth noting that the poor performance of TCP Vegas is not unique to LEO satellite networks. The text mentions that this sensitivity to network delays can lead to low throughput in other environments as well.

In [35]:
query = "Compare how TCP CUBIC and TCP Hybla address the problem of 'RTT Unfairness.'"
pose(query)

Compare how TCP CUBIC and TCP Hybla address the problem of 'RTT Unfairness.'


Retrieved documents:
CUBIC-checkpoint.pdf
CUBIC.pdf
Hybla.pdf



TCP CUBIC and TCP Hybla are two variants of the Transmission Control Protocol (TCP) that aim to address the issue of "RTT Unfairness." RTT Unfairness occurs when connections with different round-trip times (RTTs) compete for bandwidth on a shared link, leading to unfair allocation of resources.

**TCP CUBIC**

TCP CUBIC is designed to mitigate RTT Unfairness by adjusting the congestion window growth rate based on the observed RTT. When the RTT increases, TCP CUBIC reduces the congestion window growth rate to prevent the connection from consuming too much bandwidth. This approach helps to ensure that connections with longer RTTs do not starve those with shorter RTTs.

**TCP Hybla**

TCP Hybla takes a different approach to addressing RTT Unfairness. It uses a dynamic adjustment of the slow-start threshold (ssthresh) based on the observed RTT. When the RTT increases, TCP Hybla reduces the ssthresh, which in turn limits the congestion window growth rate. This approach helps to prevent connections with longer RTTs from consuming too much bandwidth and starving those with shorter RTTs.

**Comparison**

Both TCP CUBIC and TCP Hybla aim to address RTT Unfairness by adjusting the congestion window growth rate or slow-start threshold based on the observed RTT. However, they differ in their approach:

* TCP CUBIC adjusts the congestion window growth rate directly.
* TCP Hybla adjusts the slow-start threshold.

In terms of fairness and friendliness, both protocols aim to ensure that connections with different RTTs share the bandwidth fairly. However, TCP Hybla is designed to be more friendly to wireless connections, which often have longer RTTs due to the inherent delay introduced by wireless channels.

**Packet spacing**

TCP Hybla also introduces a packet spacing mechanism to reduce burstiness and improve fairness in networks with heterogeneous RTTs. This feature helps to spread out the transmission of packets over the RTT, reducing the likelihood of congestion and improving overall network performance.

In summary, while both TCP CUBIC and TCP Hybla address the issue of RTT Unfairness, they differ in their approach and design. TCP Hybla is designed to be more friendly to wireless connections and introduces a packet spacing mechanism to improve fairness and reduce burstiness.

In [36]:
query = "What is the significance of the 'recover' variable and 'Partial ACKs' in the NewReno algorithm?"
pose(query)

What is the significance of the 'recover' variable and 'Partial ACKs' in the NewReno algorithm?


Retrieved documents:
NewReno.pdf
NewReno.pdf
NewReno.pdf



The `recover` variable and partial acknowledgments play crucial roles in the NewReno algorithm for handling multiple packet losses during Fast Recovery.

**`recover` Variable:**

* The `recover` variable is initialized with the initial send sequence number.
* When three duplicate ACKs are received, the sender checks if the Cumulative Acknowledgement field covers more than the current value of `recover`. If it does, the algorithm proceeds to Step 1A (Invoking Fast Retransmit). Otherwise, it goes to Step 1B (Not invoking Fast Retransmit).
* During Fast Recovery, the highest sequence number transmitted is recorded in the `recover` variable.
* The `recover` variable serves as a threshold for determining whether a new packet loss has occurred. If an ACK arrives that acknowledges data up to and including the current value of `recover`, it indicates that all packets sent since the beginning of Fast Recovery have been received by the receiver, except for one (the next in-sequence packet).

**Partial Acknowledgments:**

* A partial acknowledgment is an ACK that acknowledges some but not all of the packets transmitted before entering Fast Retransmit.
* When a partial acknowledgment is received during Fast Recovery, the sender infers that the next in-sequence packet has been lost and retransmits it (Step 5).
* The `recover` variable helps determine whether the ACK is a partial acknowledgment or an acknowledgement for all packets sent since the beginning of Fast Recovery.

In summary, the `recover` variable and partial acknowledgments are essential components of the NewReno algorithm. They enable the sender to:

1. Determine when to invoke Fast Retransmit (Step 1A).
2. Record the highest sequence number transmitted during Fast Recovery.
3. Identify whether an ACK is a partial acknowledgment or an acknowledgement for all packets sent since the beginning of Fast Recovery.

By using these components, NewReno can more effectively handle multiple packet losses during Fast Recovery and improve network performance in scenarios where SACK is not available.

In [37]:
query = "Explain the sequential relationship between BBR’s 'Startup' and 'Drain' states."
pose(query)

Explain the sequential relationship between BBR’s 'Startup' and 'Drain' states.


Retrieved documents:
BBR.pdf
BBR.pdf
BBR.pdf



According to the text, the sequential relationship between BBR's 'Startup' and 'Drain' states is as follows:

1. The connection starts in the 'Startup' state, where it rapidly probes for bandwidth using a pacing_gain of 5/4.
2. If the startup phase is successful (i.e., the connection reaches high throughput), the connection transitions to the 'Drain' state.
3. In the 'Drain' state, the connection drains any resulting queue by reducing its pacing_gain to 1/4 for a short period of time (typically one RTprop).
4. After draining the queue, the connection leaves the 'Drain' state and enters either the 'ProbeBW' or 'Startup' states, depending on whether it estimates that the pipe was filled already.

So, in summary, the 'Drain' state is a transitional state between the 'Startup' and 'ProbeBW' states, where the connection drains any queue that may have built up during the startup phase.

___

How do we know that these answers are coming from the documents or are from the model's pre-trained knowledge? This takes a bit of prompt engineering. Let's try editing the system message:

In [41]:
@dynamic_prompt
def prompt_with_context(request: ModelRequest) -> str:
    """Inject context into state messages."""
    last_query = request.state["messages"][-1].text
    retrieved_docs = vector_store.similarity_search(last_query, k=3)

    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

    # Print documents used:
    print("\nRetrieved documents:") ##
    for doc in retrieved_docs: ##
        print(doc.metadata["source"]) ##
    print()

    system_message = (
        # Edit here:
        "You are a helpful assistant. Answer only using the information from the following documents."
        f"\n\n{docs_content}"
    )

    return system_message


agent = create_agent(llm, tools=[], middleware=[prompt_with_context])

def pose(query):
    for step in agent.stream(
        {"messages": [{"role": "user", "content": query}]},
        stream_mode="values",
    ):
        display(Markdown(step["messages"][-1].text))

In [48]:
query = "Who was the fifth president of the United States?"
pose(query)

Who was the fifth president of the United States?


Retrieved documents:
BBR.pdf
NewReno.pdf
comparative_study.pdf



The text doesn't mention the fifth president of the United States. It appears to be a research paper about congestion control schemes for LEO (Low Earth Orbit) satellite networks and their performance compared to traditional terrestrial networks. If you're looking for information on U.S. presidents, I'd be happy to help with that!